In [ ]:
import torch
import gc
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
from torch import nn
from torch import optim
import json
from PIL import Image
import io
import cv2
import torchvision.transforms.functional as TF
from torchvision.utils import save_image
from torchvision.transforms import ToPILImage
from matplotlib import pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data
import torch.optim as optim
from torch.optim import lr_scheduler
from tqdm import tqdm
import time
import os
import copy
import pickle
import urllib.request
import requests
from matplotlib.pyplot import imshow
import random
import seaborn as sns
import importlib
import pandas as pd

In [ ]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(1)
from utils import *

In [ ]:
import image_net_python

In [ ]:
#Function to display image and predicted class
def imstats(name):
    # read in image and view it
    x = readim(name, forward_normalize) 
    imshow_tensor(x[0], inv_normalize)

    #get predicted class and probability
    prob = lay2(pretrained_model(x.cuda()))
    maxcls = prob.max(1)
    print("Class is {} ({}) with confidence {}%".format(maxcls.indices.item(),class_dict[maxcls.indices.item()],round(maxcls.values.item()*100,4)))

In [ ]:
#Get classes
CLASS_URL = 'https://gist.githubusercontent.com/yrevar/6135f1bd8dcf2e0cc683/raw/d133d61a09d7e5a3b36b8c111a8dd5c4b5d560ee/imagenet1000_clsid_to_human.pkl'
class_dict = pickle.load(urllib.request.urlopen(CLASS_URL))

#Retrieve image and assign class
#IMG_URL = "https://bloximages.chicago2.vip.townnews.com/chippewa.com/content/tncms/assets/v3/editorial/c/ca/cca0d87d-8d7c-520f-b012-bc065d18b3dd/553cbf5aa0ffb.image.jpg?crop=200%2C200%2C0%2C8&resize=1200%2C1200&order=crop%2Cresize"
#IMG_URL = "https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSa3p29cIlgj0yQwMuLZzh8PKkgBUzbPLWrU-7K79DjDL498JsA&usqp=CAU"
IMG_URL = "https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ5X7yUzT_3kR7guoqCF5zEFZkesvEkagag8Y1KFJoULsXrL3h0&usqp=CAU"
#IMG_URL = "https://5.imimg.com/data5/RB/GL/IO/ANDROID-80735891/product-jpeg-500x500.jpg"
classidx = 504
response = requests.get(IMG_URL)
#img = Image.open(io.BytesIO(response.content))
img = Image.open('images/original/col_amb_up.jpg')
## for empty signal
#img = Image.new('RGB', (224, 224), (255, 255, 255))
print(img.size)
imshow(img)

In [ ]:
#Model parameters
model_img_size = 224
model_transform = transforms.Compose([transforms.Resize((model_img_size,model_img_size)),
                                         transforms.ToTensor(),
                                         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                              std=[0.229, 0.224, 0.225])])
pil_to_tensor = transforms.ToTensor()
tensor_to_pil = transforms.ToPILImage()
model_resize = transforms.Resize((224,224))


inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.225]
)
forward_normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

print(type(img))
img_input = model_transform(img)
#img_input = img_input.unsqueeze(0)
#img_input = Variable(img_input)
img_input.shape

In [ ]:
#Model definition
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pretrained_model = models.resnet101(pretrained=True)
pretrained_model.to(device)
pretrained_model.eval()

#Pass input through model
img_input = img_input.to(device)
pred = pretrained_model(img_input.unsqueeze(0))

#Use softmax to get predicted probability and view it
lay2 = torch.nn.Softmax(dim=1)
prob = lay2(pred)
maxOcls = prob.max(1)
print("Class is {} ({}) with confidence {}%".format(maxOcls.indices.item(),class_dict[maxOcls.indices.item()],maxOcls.values.item()*100))

In [ ]:
def make_prediction(img):
    #Pass input through model
    img = model_transform(img)
    img_input = img.to(device)
    pred = pretrained_model(img_input.unsqueeze(0))

    #Use softmax to get predicted probability and view it
    lay2 = torch.nn.Softmax(dim=1)
    prob = lay2(pred)
    maxOcls = prob.max(1)
    print("Class is {} ({}) with confidence {}%".format(maxOcls.indices.item(),class_dict[maxOcls.indices.item()],maxOcls.values.item()*100))

In [ ]:
def get_object_mask(input_image):
    #print(test[-3][:])
    test = inv_normalize(input_image)
    test = np.uint8(test.numpy()*255).transpose((1,2,0))
    test = 1 - test
    #test = cv2.cvtColor(test, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(test, cv2.COLOR_RGB2GRAY)
    th, thgray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY); 
    #blurred = cv2.GaussianBlur(thgray, (9, 9), 0)
    th, blurred = cv2.threshold(thgray, 0, 255, cv2.THRESH_BINARY_INV)
    edged = cv2.Canny(blurred, 1, 250, L2gradient=True)   
    #imgplot = plt.imshow(blurred, cmap="gray")
    #plt.show()
    return thgray
    #imshow(input_image[0])
    #applying closing function
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7))
    closed = cv2.morphologyEx(blurred, cv2.MORPH_CLOSE, kernel)    #finding_contours
    (cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)   
    #cnts = [] #only for plate class 
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        cv2.drawContours(closed, [approx], -1, (0, 255, 0), 2)    
    th, im_th = cv2.threshold(closed, 254, 255, cv2.THRESH_BINARY_INV);    
    h, w = im_th.shape[:2]
    im_floodfill = im_th.copy()
    mask = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(im_floodfill, mask, (0,0), 0);    # Invert floodfilled image
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)    # Combine the two images to get the foreground.
    #imshow(im_th)
    #plt.show()
    #imshow(im_floodfill_inv)
    #plt.show()
    im_out = im_th ^ im_floodfill_inv
    im_out = cv2.GaussianBlur(im_out, (3, 3), 0)
    return im_out
print(type(img_input))
imshow_tensor(img_input.cpu(),inv_normalize)
imgplot = plt.imshow(get_object_mask(img_input.cpu()))
plt.show()
print(get_object_mask(img_input.cpu()).shape)

In [ ]:
def imcorr(repla):
    newrep = torch.zeros_like(repla, dtype=torch.float)
    rp = torch.tensor([-1.25006,  2.40351, -0.15585,  0.00132])
    gp = torch.tensor([-1.09171,  2.20569, -0.11374, -0.00064])
    bp = torch.tensor([-1.04359,  1.84006,  0.21149, -0.00933])
    newrep[0] = rp[0] * torch.pow(repla[0], 3) + rp[1] * torch.pow(repla[0], 2) + rp[2] * repla[0] + rp[3]
    newrep[1] = gp[0] * torch.pow(repla[1], 3) + gp[1] * torch.pow(repla[1], 2) + gp[2] * repla[1] + gp[3]
    newrep[2] = bp[0] * torch.pow(repla[2], 3) + bp[1] * torch.pow(repla[2], 2) + bp[2] * repla[2] + bp[3]
    return newrep

img_dir = "./images/original/mug/"
image_list = [Image.open(img_dir+x) for x in os.listdir(img_dir)]
train_image_size = 252
image_tensors = torch.cat([
                pil_to_tensor(
                    x.resize((train_image_size,train_image_size)))
                     .unsqueeze(0) for x in image_list])
image_tensors = image_tensors.to(device)

In [ ]:
def get_polynomial(margin):
    x = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    out = []
    for i in range(3):
        y = [max(0, xi + random.random()*2*margin - margin) for xi in x]
        out.append(np.polyfit(x,y,3))
    return out
def imcorr2(repla, coeff):
    newrep = torch.zeros_like(repla, dtype=torch.float)
    rp = coeff[0]
    gp = coeff[1]
    bp = coeff[2]
    newrep[0] = rp[0] * torch.pow(repla[0], 3) + rp[1] * torch.pow(repla[0], 2) + rp[2] * repla[0] + rp[3]
    newrep[1] = gp[0] * torch.pow(repla[1], 3) + gp[1] * torch.pow(repla[1], 2) + gp[2] * repla[1] + gp[3]
    newrep[2] = bp[0] * torch.pow(repla[2], 3) + bp[1] * torch.pow(repla[2], 2) + bp[2] * repla[2] + bp[3]
    return newrep

In [ ]:
x = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
y = [max(0, xi + random.random()*2*0.2 - 0.2) for xi in x]
coeffs = np.polyfit(x,y,3)[::-1]
print(coeffs[::-1])
y = np.array([np.sum(np.array([coeffs[i]*(j**i) for i in range(len(coeffs))])) for j in x])
plt.plot(x, y)
plt.show()

In [ ]:
def get_image_batch(batch_size, train_image_size):
    selected_images = indices = torch.randperm(len(image_list))[:batch_size]
    return image_tensors[selected_images]

In [ ]:
account_resize = True

In [ ]:
img_full = Image.open('images/original/col_full_up.jpg')

In [ ]:
importlib.reload(image_net_python)

In [ ]:
arg_list = [
    {
        'device':device,
        'exposure':2000,
        'batch_size':32,
        'image_cache_size':1,
        'original_images':[img, img_full],
        'number_test_images':1,
        'n_epochs':1,
        'classes_to_skip':[504, 901, 968, 647,899,725,441,505,441,969,438,967],
        'classidx':504,
        'model_img_size':252,
        'targidx':722,
        'apply_transformations':False,
        'ambient_light':1,
        'color_noise':None
    },
    {
        'device':device,
        'exposure':2000,
        'batch_size':32,
        'image_cache_size':1,
        'original_images':[img, img_full],
        'number_test_images':1,
        'n_epochs':10000,
        'classes_to_skip':[504, 901, 968, 647,899,725,441,505,441,969,438,967],
        'classidx':504,
        'model_img_size':252,
        'targidx':722,
        'apply_transformations':False,
        'ambient_light':1,
        'color_noise':None
    },
    {
        'device':device,
        'exposure':2000,
        'batch_size':32,
        'image_cache_size':1,
        'original_images':[img, img_full],
        'number_test_images':1,
        'n_epochs':10000,
        'classes_to_skip':[504, 901, 968, 647,899,725,441,505,441,969,438,967],
        'classidx':504,
        'model_img_size':252,
        'targidx':722,
        'apply_transformations':False,
        'ambient_light':1,
        'color_noise':"gy_noise"
    }
]

In [ ]:
output_values = []
for a in range(len(arg_list)):
    args = arg_list[a]
    args_in_list = []
    for v in args.values(): args_in_list.append(v)
    args_in_list.append(args)
    max_acc, class_acc, target_acc, targloss, origloss, targidx = image_net_python.run_simulation(*args_in_list)
    '''
    #View original loss and target loss
    plt.plot(targloss, label="target")
    plt.plot(origloss, label="original")
    plt.xlabel('no of iterations/10',fontsize=13)
    plt.ylabel('loss',fontsize=13)
    plt.legend()
    plt.show()
    plt.figure(figsize=(10,5))
    chart = sns.countplot([class_dict[x[0]][:15] for x in max_acc])
    chart.set_xticklabels(chart.get_xticklabels(), rotation=90)
    '''
    avg_class_acc = np.array(class_acc).mean()
    avg_target_acc = np.array(target_acc).mean()
    dodge_success = np.array([1 if x[0] not in args['classes_to_skip'] else 0 for x in max_acc]).mean()
    target_success = np.array([1 if x[0]==targidx else 0 for x in max_acc]).mean()
    output_classes = {}
    for cl in max_acc: 
        if cl[0] not in output_classes: output_classes[cl[0]]=0
        output_classes[cl[0]]+=1
    print("Avg. accuracy for {} (original) - {}".format(class_dict[args['classidx']],avg_class_acc))
    print("Avg. accuracy for {} (target)- {}".format(class_dict[targidx],avg_target_acc))
    print("Dodge success outside classes to skip - {}".format(dodge_success))
    print("Target success - {}".format(target_success))
    print("Output class histogram - ",output_classes)
    output_values.append((avg_class_acc,avg_target_acc,dodge_success,target_success,output_classes))

output_values_config = ["Class acc", "Target acc", "Dodge(similar)","Targeted"]
input_configs = [
    {'base':'case'},
    {'noise':"No"},
    {'noise':"gy_noise"}
]

df_list = []
for i,inpc in enumerate(input_configs):
    config_name = ",".join(["{}-{}".format(k,v) for k,v in inpc.items()])
    for j in range(4):
        df_list.append([config_name, output_values[i][j], output_values_config[j]])
df_results = pd.DataFrame(df_list, columns = ['config','metric_value','metric_type'])

plot_title = 'Different noise - 10k, no transformation'
sns.lineplot(data=df_results, x='config', y='metric_value',hue="metric_type",sort=False).set_title(plot_title)
plt.xticks(rotation=30)
#plt.savefig('plots/'+plot_title+'.pdf')


In [ ]:
imshow(img_full)

In [ ]:
def aug_transform(image1, image2, brightness_multiplier=None):
        # Adjust Brightness
        if brightness_multiplier==None:
            brightness_multiplier = 1 + random.random()
        imageb = TF.adjust_brightness(image1, brightness_multiplier)
    
        # Resize
        re_size = random.randint(252,360)
        resize = transforms.Resize(re_size)
        image1 = resize(image1)
        image2 = resize(image2)
        imageb = resize(imageb)

        # Random crop
        i, j, h, w = transforms.RandomCrop.get_params(
            image1, output_size=(252, 252))
        image1 = TF.crop(image1, i, j, h, w)
        image2 = TF.crop(image2, i, j, h, w)
        imageb = TF.crop(imageb, i, j, h, w)

        # Random horizontal flipping
        if random.random() > 0.5:
            image1 = TF.hflip(image1)
            image2 = TF.hflip(image2)
            imageb = TF.hflip(imageb)

        # Random vertical flipping
        if random.random() > 0.5:
            image1 = TF.vflip(image1)
            image2 = TF.vflip(image2)
            imageb = TF.vflip(imageb)
            
        # Random Rotation
        rotate = transforms.RandomRotation(45, resample=Image.BILINEAR)
        seed = random.randint(0,2**32)
        random.seed(seed)
        image1 = rotate(image1)
        random.seed(seed)
        image2 = rotate(image2)
        random.seed(seed)
        imageb = rotate(imageb)

        return image1, imageb, image2

In [ ]:
img_temp, img_br, img_full_temp = aug_transform(img, img_full, 1.5)
imshow(img_br)

In [ ]:
imshow(img_full_temp)

In [ ]:
make_prediction(img_full_temp)

In [ ]:
def resize2d(img, size):
        return (F.adaptive_avg_pool2d(Variable(img), size)).data
def upsample2d(img, size=224):
    upsample = nn.Upsample(size=size, mode='bilinear', align_corners=False)
    return upsample(torch.unsqueeze(img, 0))[0]

def get_image_pair(img, img_full):
    repeat_size = int(3024/3024)
    img = img.resize((252,252))
    model_img_size = img.size[0]
    img_t, img_f = aug_transform(img, img_full)
    img_t = pil_to_tensor(img_t)
    img_t = img_t.to(device)
    img_f = pil_to_tensor(img_f.resize((252,252))).to(device)
    return img_t, img_f

def get_image_trip(img, img_full):
    repeat_size = int(3024/3024)
    img = img.resize((252,252))
    model_img_size = img.size[0]
    img_t, img_b, img_f = aug_transform(img, img_full, 1)
    img_t = pil_to_tensor(img_t)
    img_t = img_t.to(device)
    img_b = pil_to_tensor(img_b).to(device)
    img_f = pil_to_tensor(img_f.resize((252,252))).to(device)
    return img_t, img_b, img_f

#For resize post convolution
if account_resize:
    repeat_size = int(3024/3024)
    img = img.resize((252,252))
    model_img_size = img.size[0]
    img_t = pil_to_tensor(img)
    img_t = img_t.to(device)
    img_f = pil_to_tensor(img_full.resize((252,252))).to(device)
else:
    model_img_size = 224

In [ ]:
cache_size = 150
image_cache = []
for i in range(cache_size):
    image_cache.append(get_image_trip(img, img_full))

In [ ]:
# typical exposure is in form 1/n s. Available: 15, 20, 25 30, 40, 50, 60, 80, 100, 125, 160, 200, 250
exposure = 500 
exp_micros = 1000000/exposure          # get exposure in microseconds
img_ratio = 3024 / model_img_size      # every row in model is img_ratio rows in original image
model_tr = 10 * img_ratio              # multiply real tr (10 micros) by img_ratio to find model tr
conv_size = exp_micros / model_tr      # divide exposure time by tr to find convolution size
conv_size = int(conv_size)             # Need closest integer approximation. Won't cause a significant difference
conv_size

In [ ]:
sz = model_img_size + conv_size - 1 # 300   #Length of input signal
c = 0    #Ambient light ratio
c_limits = [0,0]
batch = 16
channels = 3
# change of variable term to optimise on
w = torch.rand([channels,sz,1], requires_grad=True, dtype=torch.float, device=device)

#Create the mask to only illuminate the object
mask = torch.tensor(get_object_mask(img_input.cpu()), dtype=torch.float, device=device)
mask = mask / torch.max(mask)


#Target and original class labels
orig = torch.tensor([classidx], dtype=torch.long, device=device)

#Model parameters
lr = 1e-1
n_epochs = 5
#optimizer = optim.SGD([w], lr=lr, momentum=0.9, nesterov=True)
optimizer = optim.Adam([w], lr=0.01)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
n_epochs = 10000
classes_to_skip = [classidx, 901, 968, 647,899,725,441,505,441,969,438,967]
targidx = 722
target = torch.tensor([targidx], dtype=torch.long, device=device)

In [ ]:
#Track the loss to target and original class
targloss = []
origloss = []

#obj_dict = {}

#Optimisation loop. initially untargeted
for epoch in tqdm(range(n_epochs)):
    
    #Switch to targeted at halfway point
    half = epoch < n_epochs//6
    if epoch == n_epochs//6:
        tops = out.topk(40).indices[0]
        for t in tops:
            if t.item() not in classes_to_skip: 
                targidx = t.item()
        #targidx = 722
        #targidx = tops[0].item() if tops[0].item() != classidx else tops[1].item()
        target = torch.tensor([targidx], dtype=torch.long, device=device)
        print("Switching from untarget to target {}".format(targidx))
    
    if channels==1:
        n_w = w.repeat(3,1,1)
    else:
        n_w = w
    
    # For resize post convolution
    if account_resize:
        n_w = torch.repeat_interleave(n_w, repeats=repeat_size, dim=1)
    
    sig_height = model_img_size + conv_size - 1
    gy, new_w, sh = fttogy(n_w, batch, None, c_limits, sig_height, conv_size, device, 0, shifting=True)
    #display(tensor_to_pil(gy[0]))
    #color correction
    #gy = torch.cat([imcorr(i).unsqueeze(0) for i in gy])
    
    #gamma correction
    #gy = torch.pow(gy, 1/2.2)
    
    if account_resize:
        #For resize post convolution
        #inp = gy*img_t
        #gy *= random.random()*0.7 + 0.7
        
        #Add noise to input signal
        #xx = 0.4*torch.rand((gy.size()[0],gy.size()[1]),device=device)-0.2
        #xx = xx.unsqueeze(2).unsqueeze(3).repeat(1,1,252,1)
        #gy += xx
        #gy = torch.clamp(gy, min=0, max=1)
        
        # without brightness random
        #img_t, img_f = image_pair_cache[random.randint(0,cache_size-1)]
        #inp = torch.pow(0.0000001 + torch.pow(img_t,2.2) + gy*(torch.pow(img_f,2.2)-torch.pow(img_t,2.2)), 1/2.2)
        
        img_amb, img_bright, img_f = img_t, img_t, img_f#image_cache[random.randint(0,cache_size-1)]
        inp = torch.pow(0.0000001 + torch.pow(img_bright,2.2) + gy*(torch.pow(img_f,2.2)-torch.pow(img_amb,2.2)), 1/2.2)
        
        #For multiple train images
        #inp = gy*get_image_batch(batch, 252)
    else: 
        inp = gy*inv_normalize(img_input)
    
    #Gaussian Noise
    #inp = inp + torch.randn(inp.size(),device=device)*0.005
    #inp = torch.clamp(inp,min=0,max=1)
    
    #Uniform noise to each row
    #inp = inp*(1 + torch.rand(inp.size()[:-1],device=device).unsqueeze(-1)*0.4 - 0.2)
    #inp = torch.clamp(inp,min=0,max=1)
    
    #apply random color correction
    #inp = torch.cat([imcorr2(i, get_polynomial(0.2)).unsqueeze(0) for i in inp])
    
    if account_resize:
        #For resize post convolution
        inp = torch.cat([forward_normalize(upsample2d(i,224)).unsqueeze(0) for i in inp])
    else: 
        inp = torch.cat([forward_normalize(i).unsqueeze(0) for i in inp])
    inp = inp.to(device)
    out = pretrained_model(inp)

    #Calculate Loss depended on if targeted or untargeted
    if not half: targLoss = loss_fn(out, target.repeat(batch))
    origLoss = loss_fn(out, orig.repeat(batch))
    loss = -origLoss if half else targLoss
    if epoch%10 == 0:
        targloss.append(0 if half else targLoss)
        origloss.append(origLoss)
    if epoch%1000 == 0:
        if not half: print(targLoss, origLoss) 
        else: print(origLoss)
    loss.backward()   
    
    if epoch%10==0:   # batch 32 === update after 10 for ~224
        optimizer.step()
        optimizer.zero_grad()
        del loss
        torch.cuda.empty_cache()
    
    if epoch!=n_epochs-1:
        del inp
        del new_w
    #else:
        #saving w to be used for prediction
        #torch.save(n_w,'w_0.5_764.pt')

    #Code to check gpu allocation    
    '''
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                #print(type(obj), obj.size())
                if type(obj) not in obj_dict:
                    obj_dict[type(obj)] = 1
                else:
                    obj_dict[type(obj)] += 1
        except: pass
    print(obj_dict)
    obj_dict.clear()
    '''
    
#View original loss and target loss
plt.plot(targloss, label="target")
plt.plot(origloss, label="original")
plt.xlabel('no of iterations/10',fontsize=13)
plt.ylabel('loss',fontsize=13)
plt.legend()
plt.show()

In [ ]:
#Create softmax layer to view probabilities
lay2 = torch.nn.Softmax(dim=1)
#targidx = 4
prob = lay2(out)
maxcls = prob.max(1)
#print(maxcls)
for i in range(batch):
    #print("alpha: {}".format(c[i].item()))
    print("target {}: {}%".format(targidx,prob[i][targidx]))
    print("Orig {}: {}%".format(classidx,prob[i][classidx]))
    print("Class is {} ({}) with confidence {}%\n".format(maxcls.indices[i].item(),class_dict[maxcls.indices[i].item()],maxcls.values[i].item()*100))

In [ ]:
classidx = 504
print('source :'+str(np.array([float(prob[i][classidx]) for i in range(batch) if prob.max(1).indices[i] == classidx]).mean()))
print('target :'+str(np.array([float(prob[i][722]) for i in range(batch)]).mean()))
print('max :'+str(np.array([float(prob[i][prob.max(1).indices[i].item()]) for i in range(batch)]).mean()))

In [ ]:
#torch.save(new_w,'saved_inputs/imagenet_iso800_504_targeted722_rgb_resize252_spatialtransforms.pt')
test = torch.ones([3,252,252])
imshow_tensor(forward_normalize(gy[15].detach().cpu()*test), inv_normalize)

In [ ]:
#View input image and adversarial image. Save both
#imshow_tensor(forward_normalize(inv_normalize(img_input.cpu())*c), inv_normalize)
imshow_tensor(inp[5].detach().cpu(),inv_normalize)
imshow_tensor(inp[10].detach().cpu(),inv_normalize)
imshow_tensor(inp[15].detach().cpu(),inv_normalize)
#imshow_tensor(inp[20].detach().cpu(),inv_normalize)
#imshow_tensor(inp[25].detach().cpu(),inv_normalize)
#saveim(img_input.cpu(), "original_{}.png".format(classidx),inv_normalize)
#saveim(forward_normalize(inv_normalize(img_input.cpu())*c), "original_c{}_{}.png".format(c,classidx),inv_normalize)
#saveim(inp, "src{}_c{}_sz{}_tg{}.png".format(classidx,c,sz,targidx),inv_normalize)

In [ ]:
plt.plot(torch.flatten(new_w[0][0]).detach().cpu(), label="w_r")
plt.legend()
plt.show()
plt.plot(torch.flatten(new_w[0][1]).detach().cpu(), label="w_g")
plt.legend()
plt.show()
plt.plot(torch.flatten(new_w[0][2]).detach().cpu(), label="w_b")
plt.legend()
plt.show()

In [ ]:
#evaluate for different augmented images
class_acc, target_acc, max_acc = [],[],[]
obj_dict = {}
for i in range(1):
    # without random bright
    #img_t2, img_f2 = get_image_pair(img, img_full)
    #inp2 = torch.pow(0.0000001 + torch.pow(img_t2,2.2) + gy*(torch.pow(img_f2,2.2)-torch.pow(img_t2,2.2)), 1/2.2)
    
    img_t2, img_b2, img_f2 = img_t, img_t, img_f#get_image_trip(img, img_full)
    inp2 = torch.pow(0.0000001 + torch.pow(img_b2,2.2) + gy*(torch.pow(img_f2,2.2)-torch.pow(img_t2,2.2)), 1/2.2)
    
    inp2 = torch.cat([forward_normalize(upsample2d(i,224)).unsqueeze(0) for i in inp2])
    inp2 = inp2.to(device)
    out2 = pretrained_model(inp2)
    prob2 = lay2(out2)
    maxcls2 = prob2.max(1)
    #print(maxcls)
    for i in range(batch):
        #print("alpha: {}".format(c[i].item()))
        #print("target {}: {}%".format(targidx,prob2[i][targidx]))
        target_acc.append(prob2[i][targidx].item())
        class_acc.append(prob2[i][classidx].item())
        #print("Orig {}: {}%".format(classidx,prob2[i][classidx]))
        max_acc.append((maxcls2.indices[i].item(), prob2[i][maxcls2.indices[i].item()].item()))
        #print("Class is {} ({}) with confidence {}%\n".format(maxcls2.indices[i].item(),class_dict[maxcls2.indices[i].item()],maxcls2.values[i].item()*100))
    del img_t2
    del img_f2
    del inp2
    del out2
    del prob2
    del maxcls2
    torch.cuda.empty_cache()
plt.figure(figsize=(10,5))
chart = sns.countplot([class_dict[x[0]][:15] for x in max_acc])
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)

In [ ]:
type(img_full)

In [ ]:
#evaluate for different augmented images without signal
class_acc, target_acc, max_acc = [],[],[]
obj_dict = {}
for i in range(50):
    img_t2, img_f2 = get_image_pair(img, img_full)
    inp2 = forward_normalize(upsample2d(img_t2,224)).unsqueeze(0)
    inp2 = inp2.to(device)
    out2 = pretrained_model(inp2)
    prob2 = lay2(out2)
    maxcls2 = prob2.max(1)
    
    target_acc.append(prob2[0][targidx].item())
    class_acc.append(prob2[0][classidx].item())
    #print("Orig {}: {}%".format(classidx,prob2[i][classidx]))
    max_acc.append((maxcls2.indices[0].item(), prob2[0][maxcls2.indices[0].item()].item()))
    #print("Class is {} ({}) with confidence {}%\n".format(maxcls2.indices[i].item(),class_dict[maxcls2.indices[i].item()],maxcls2.values[i].item()*100))
    del img_t2
    del img_f2
    del inp2
    del out2
    del prob2
    del maxcls2
    torch.cuda.empty_cache()
plt.figure(figsize=(5,5))
chart = sns.countplot([class_dict[x[0]][:15] for x in max_acc])
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)

In [ ]:
imstats("original_919.png")
imstats("original_c0.5_919.png")
imstats("original_c0.3_919.png")

In [ ]:
#Get classes
CLASS_URL = 'https://gist.githubusercontent.com/yrevar/6135f1bd8dcf2e0cc683/raw/d133d61a09d7e5a3b36b8c111a8dd5c4b5d560ee/imagenet1000_clsid_to_human.pkl'
class_dict = pickle.load(urllib.request.urlopen(CLASS_URL))

#Retrieve image and assign class
IMG_URL = "https://bloximages.chicago2.vip.townnews.com/chippewa.com/content/tncms/assets/v3/editorial/c/ca/cca0d87d-8d7c-520f-b012-bc065d18b3dd/553cbf5aa0ffb.image.jpg?crop=200%2C200%2C0%2C8&resize=1200%2C1200&order=crop%2Cresize"
classidx = 919
response = requests.get(IMG_URL)
img = Image.open(io.BytesIO(response.content))
print(img.size)
imshow(img)

In [ ]:
test_length = 5
test = torch.tensor([[0., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3.]],device=device)
print(test)
test_batch = 2
test_shifted = shift_operation(test.unsqueeze(0).repeat(test_batch,1,1,1).view(-1, test_length, 1), [0,1]).view(test_batch,3,test_length,1)
print(test_shifted.shape)
print(test_shifted)
test_reshaped = test_shifted.view([3,1,test_length,test_batch])
treshape = test_shifted.transpose(0,3).transpose(0,1)
print(test_reshaped.shape)
print(test_reshaped)
print(treshape)

In [ ]:
a = torch.rand((8,8),dtype=torch.float)
import math
pi = math.pi
print(a)
print(torch.round(a*45)/45)
print(torch.mean(torch.abs(a - torch.round(a*45)/45)))
b = a-torch.sin(90*pi*a)/(90*pi)
print(b)
print(torch.mean(torch.abs(b - torch.round(a*45)/45)))
c = b-torch.sin(90*pi*b)/(90*pi)
print(c)
print(torch.mean(torch.abs(c - torch.round(a*45)/45)))
d = c-torch.sin(90*pi*c)/(90*pi)
print(d)
print(torch.mean(torch.abs(d - torch.round(a*45)/45)))